# Search HERUS sample (Clements+2018) observed lines in ALMA archive
## 1. Search semi-manually in the notebook.
## 2. Do a comparison search with `runQueryWithLines` method

### <span style="color:red">Known issues are written in red</span>

In [1]:
from astropy.table import Table
# read in sample
HERUS_sample = Table.read('HERUS_sample.fits')


Tidy up trailing white space in the name column (helps with matching later)

In [2]:
names = [name.strip() for name in HERUS_sample['name']]

HERUS_sample['name'] = names

In [3]:
HERUS_sample

name,ra,dec,z,log(LIR/L),ObsID_C18,F250,F350,F500,F250_err,F350_err,F500_err,Temp(K),T+,T-,Beta,B+,B-,A,A+,A-,DustMass(logMsol),HERUS
str15,float64,float64,float32,float32,bytes11,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes6,float32,float32,float32,bool
IRAS 00397-1312,10.5647,-12.93412,0.262,12.97,1342234696,0.389,0.13,0.04,0.004,0.004,0.005,41.73,1.19,1.19,2.03,0.08,0.08,- 1.21,0.05,0.05,8.9,True
3C273,187.27792,2.05239,0.158,12.72,1342234882,0.437,0.633,0.994,0.004,0.004,0.005,nan,nan,nan,nan,nan,nan,,nan,nan,nan,True
IRAS 20100-4156,303.37308,-41.79303,0.13,12.63,1342230817,1.001,0.349,0.102,0.004,0.004,0.005,39.51,0.4,0.39,1.96,0.03,0.03,- 0.63,0.02,0.02,8.6,True
Mrk1014,29.95921,0.39462,0.163,12.61,1342237540,0.46,0.175,0.063,0.004,0.004,0.005,46.89,1.1,1.14,1.5,0.07,0.06,- 1.30,0.04,0.04,8.4,True
IRAS 03521+0028,58.6758,0.61761,0.152,12.56,1342239850,0.684,0.27,0.094,0.004,0.004,0.004,42.62,1.15,1.27,1.57,0.05,0.05,- 1.02,0.04,0.04,8.6,True
IRAS 03158+4227,49.80167,42.64111,0.134,12.55,1342226656,0.973,0.377,0.137,0.004,0.004,0.005,42.3,1.26,1.37,1.58,0.04,0.04,- 0.84,0.04,0.04,8.6,True
IRAS 16090-0139,242.91847,-1.78516,0.134,12.54,1342229565,1.067,0.404,0.116,0.004,0.004,0.005,37.5,0.43,0.44,1.84,0.03,0.03,- 0.60,0.02,0.02,8.7,True
Mrk231,194.05931,56.87368,0.042,12.49,1342201218a,5.618,2.011,0.615,0.019,0.008,0.008,36.87,0.41,0.43,1.89,0.02,0.02,0.26,0.02,0.02,8.3,True
IRAS 00188-0856,5.36051,-8.65722,0.128,12.47,1342234693,0.877,0.345,0.111,0.004,0.004,0.005,35.76,0.88,1.05,1.74,0.05,0.05,- 0.67,0.05,0.04,8.6,True


### Set path to custom `astroquery` module
This is a temporary hack to "use a version of ```astroquery``` that does not have a bug in the release date column. 
To get this set up run the following in terminal:

`git clone https://github.com/astropy/astroquery.git`

`cd astroquery`

`python setup.py build`

`python setup.py install`

replace `python` with `python3` if necessary

In [4]:
import sys
sys.path = ['/Users/thbrown/astroquery'] + sys.path

### Import the `ALMAxmatch` tool
requires adding to the python path for now

In [5]:
import sys
sys.path = ['/Users/thbrown/ALMA/think_tank_code/tools/archiveDev'] + sys.path
from ALMAxmatch import archiveSearch

## 1. Run the archive search
1. Create list of tuples containing `SkyCoord` objects and search radii ie. (SkyCoord(ra,dec), searchRadius)*
2. Pass tuple to array along with search radius to `runQueries`, selecting only science data
3. Copy the results to a new dictionary and clean up


*see potential improvements note below

In [6]:
from astropy import units as u
from astropy.coordinates import SkyCoord

# archive search impact parameter
searchradius = '20arcsec'

# SkyCoord object
herusCoords = [(SkyCoord(ra=ra*u.degree, dec=dec*u.degree), searchradius)
                   for ra, dec in zip(HERUS_sample['ra'], HERUS_sample['dec'])]

Choose a search radius and generate target list in `herusQuery`

### Potential improvements to the search

1. Currently `survey` only takes either a tuple ('ra dec', 'radius') or lists of tuples. I don't think this is the best way for lists to operate, it would be good to be able to pass a list of positions (or `SkyCoord` object with multiple positions) and a radius.

In [7]:
# search using SkyCoord array
herusQuery = archiveSearch(herusCoords)

display the target dictionary

In [8]:
display(herusQuery.targets)

{'coord=(10.5647 deg -12.93412 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (10.5647, -12.93412)>, '20arcsec'),
 'coord=(187.27792 deg 2.05239 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (187.27792, 2.05239)>, '20arcsec'),
 'coord=(303.37308 deg -41.79303 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (303.37308, -41.79303)>, '20arcsec'),
 'coord=(29.95921 deg 0.39462 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (29.95921, 0.39462)>, '20arcsec'),
 'coord=(58.6758 deg 0.61761 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (58.6758, 0.61761)>, '20arcsec'),
 'coord=(49.80167 deg 42.64111 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (49.80167, 42.64111)>, '20arcsec'),
 'coord=(242.91847 deg -1.78516 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (242.91847, -1.78516)>, '20arcsec'),
 'coord=(194.05931 deg 56.87368 deg) radius=20arcsec': (<SkyCoord (ICRS): (r

Run the query, pulling only science data

In [9]:
herusQuery.runQueries(science=True)

The Archive frequence column is horrible to read, let's make it more human.

In [10]:
herusQuery.parseFrequencyRanges()

Now we have a dictionary `.queryResults` that has astropy tables for each target, even the null results. 

In [11]:
n = 0 # counter
for target in herusQuery.targets:
    display(herusQuery.queryResults[target])
    if len((herusQuery.queryResults[target]))>0: # only count targets with archival data
        n = n+1


Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64


Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2016.1.01073.S,3C273,187.2778539638189,2.052425419153435,289.9507708791571,64.35995216172215,[6],0.6366613102530091,62493.24686378306,12m,,6531.758,2018-03-03,"[222.99..224.96GHz,62493.29kHz,574.4uJy/beam@10km/s,35.1uJy/beam@native, XX XY YY] U [224.99..226.96GHz,62493.29kHz,589.3uJy/beam@10km/s,36.2uJy/beam@native, XX XY YY] U [238.99..240.96GHz,62493.29kHz,586.9uJy/beam@10km/s,37.1uJy/beam@native, XX XY YY] U [240.99..242.96GHz,62493.29kHz,592.6uJy/beam@10km/s,37.6uJy/beam@native, XX XY YY]",77103.74004787063,XX XY YX YY,2016-12-20T09:33:13,"Hovatta, Talvikki",3C273_a_06_TM1,"Savolainen, Tuomas; O'Sullivan, Shane; Tchekhovskoy, Alexander;",0.5743872343921695,0.01823828560997868,1.3283782,uid://A001/X87a/X885,uid://A001/X87a/X886,uid://A002/Xbbf310/X1d61,Probing the magnetic fields at the base of relativistic jets through Faraday rotation observations,S,TARGET,27.04176102625552,3.2923768678479095,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Outflows, jets, feedback",Active galaxies,2016.1.01073.S,"[array([222.99, 224.96]), array([224.99, 226.96]), array([238.99, 240.96]), array([240.99, 242.96])]"
2016.1.01216.V,3C273,187.27785368739794,2.052443798078459,289.95076175698233,64.35997011387919,[3],3.2590125179667355,15624.786288841999,12m,,6096.384,2018-10-21,"[85.33..87.20GHz,15624.96kHz,843.4uJy/beam@10km/s,33uJy/beam@native, XX XY YY] U [87.33..89.20GHz,15624.96kHz,826.2uJy/beam@10km/s,32.7uJy/beam@native, XX XY YY] U [97.39..99.26GHz,15624.96kHz,776.2uJy/beam@10km/s,32.5uJy/beam@native, XX XY YY] U [99.33..101.20GHz,15624.96kHz,772.6uJy/beam@10km/s,32.6uJy/beam@native, XX XY YY]",46285.76402485768,XX XY YX YY,2017-04-04T00:24:39,"Akiyama, Kazunori",vexed 4 April 3C273_a_03_TM1,"Fish, Vincent; Asada, Keiichi; Johnson, Michael; Gomez, Jose L.; Pu, Hung-Yi; Nagai, Hiroshi; Nakamura, Masanori; Honma, Mareki; Hada, Kazuhiro; Blackburn, Lindy; Doeleman, Sheperd; Mizuno, Yosuke; Koyama, Shoko; Chael, Andrew; Lonsdale, Colin; Marscher, Alan; Gabuzda, Denise; Bouman, Katherine;",0.7726293315855453,0.016358689847191233,0.68157107,uid://A001/X11b3/Xa,uid://A001/X11b3/Xd,uid://A002/Xbeb207/X79e,Probing the active collimation region of the relativistic jet in 3C 273,V,APPPHASE_ACTIVE TARGET,67.54820924137793,20.45384637583945,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Outflows, jets, feedback",Active galaxies,2016.1.01216.V,"[array([85.33, 87.2 ]), array([87.33, 89.2 ]), array([97.39, 99.26]), array([ 99.33, 101.2 ])]"
2018.1.00357.S,3C273,187.27785012674357,2.0524471126437542,289.95075216751826,64.35997264960277,[6],0.26552431459364534,62495.214616261044,12m,,3084.48,2019-12-24,"[223.00..224.97GHz,62495.54kHz,855.5uJy/beam@10km/s,52.3uJy/beam@native, XX XY YY] U [225.00..226.97GHz,62495.54kHz,879uJy/beam@10km/s,54uJy/beam@native, XX XY YY] U [239.00..240.97GHz,62495.54kHz,874.2uJy/beam@10km/s,55.3uJy/beam@native, XX XY YY] U [241.00..242.97GHz,62495.54kHz,883.3uJy/beam@10km/s,56.1uJy/beam@native, XX XY YY]",77106.16784740346,XX XY YX YY,2018-11-11T13:42:10,"Hovatta, Talvikki",3C273_a_06_TM1,"Savolainen, Tuomas; Marti-Vidal, Ivan; O'Sullivan, Shane; Tchekhovskoy

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2013.1.00659.S,IRAS_20100-4156,303.372954,-41.793189,358.763123154025,-32.43952353691461,[3],1.4766248761805358,976.4941902735678,12m,,12670.56,2015-09-27,"[99.20..101.19GHz,31247.92kHz,664.6uJy/beam@10km/s,27.2uJy/beam@native, XX YY] U [101.11..102.99GHz,976.50kHz,665.5uJy/beam@10km/s,28.4uJy/beam@native, XX YY]",2842.375726278533,XX YY,2014-06-09T09:25:29,"Spoon, Henrik",IRAS_201_a_03_TE,"Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Afonso, Jose; Gonzalez-Alfonso, Eduardo; Pearson, Chris; Farrah, Duncan; Bernard-Salas, Jeronimo;",0.6646029848565244,0.019617680035142075,0.7210153,uid://A001/X121/X2b1,uid://A001/X121/X2b2,uid://A002/X83b318/Xcda,Detecting in CO(1-0) the Strongest Molecular Outflow found by Herschel in the Southern Sky,S,TARGET WVR,62.31809481412121,12.646461432597606,Y,2,"Merging and interacting galaxies, Outflows, jets, feedback",Galaxy evolution,2013.1.00659.S,"[array([ 99.2 , 101.19]), array([101.11, 102.99])]"
2016.1.00615.S,IRAS20100-4156,303.37308750002956,-41.7931175000075,358.7632252909276,-32.43961064892275,[6],0.5799155173055671,7813.039344573977,12m,,5201.28,2018-01-28,"[215.86..217.73GHz,7813.05kHz,627uJy/beam@10km/s,38.9uJy/beam@native, XX YY] U [229.65..231.52GHz,7813.05kHz,630.3uJy/beam@10km/s,40.4uJy/beam@native, XX YY]",10117.512051340444,XX YY,2016-11-10T23:16:56,"Gowardhan, Avani",IRAS2010_a_06_TM1,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",0.6270057497181435,0.028027108446736643,1.912678,uid://A001/X87a/X852,uid://A001/X87a/X853,uid://A002/Xba6edb/X5b4f,Probing Dense Gas Physics in the Most Extreme Southern Molecular Outflow,S,TARGET,28.163672820080123,3.0889484522108206,Y,0,"Outflows, jets, feedback, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2016.1.00615.S,"[array([215.86, 217.73]), array([229.65, 231.52])]"
2016.1.00615.S,iras20100-4156,303.3730874999803,-41.7931175000082,358.76322529091993,-32.439610648886585,[7],0.6295592299396062,15623.376506426828,12m,,8799.84,2018-06-23,"[302.56..304.42GHz,15623.38kHz,665.6uJy/beam@10km/s,48.9uJy/beam@native, XX YY] U [312.85..314.71GHz,15623.38kHz,694.3uJy/beam@10km/s,51.9uJy/beam@native, XX YY] U [314.86..315.79GHz,15623.38kHz,717.4uJy/beam@10km/s,76uJy/beam@native, XX YY] U [315.29..316.22GHz,15623.38kHz,764.7uJy/beam@10km/s,81uJy/beam@native, XX YY]",14810.402563980124,XX YY,2017-05-01T07:26:05,"Gowardhan, Avani",iras2010_a_07_TM1,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",0.6655851575481608,0.02994593449700171,0.75314915,uid://A001/X87a/X856,uid://A001/X87a/X857,uid://A002/Xbfd686/X1571,Probing Dense Gas Physics in the Most Extreme Southern Molecular Outflow,S,TARGET,20.362940126102963,3.192691776160348,Y,0,"Outflows, jets, feedback, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2016.1.00615.S,"[array([302.56, 304.42]), array([312.85, 314.71]), array([314.86, 315.79]), array([315.29, 316.22])]"
2018.1.00065.S,iras_20100-4156,303.3731333333849

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]"
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64
2013.1.00535.S,Mrk1014,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[8],0.20453740001916954,7812.167591985681,12m,,120.96,2016-08-10,"[455.89..457.76GHz,7812.17kHz,11.7mJy/beam@10km/s,1.1mJy/beam@native, XX YY] U [456.94..458.81GHz,7812.17kHz,10.9mJy/beam@10km/s,985.3uJy/beam@native, XX YY] U [467.88..469.86GHz,31248.67kHz,11.4mJy/beam@10km/s,1mJy/beam@native, XX YY] U [469.15..471.13GHz,31248.67kHz,15.2mJy/beam@10km/s,1.3mJy/beam@native, XX YY]",5104.331575671292,XX YY,2015-06-06T12:23:46,"Rangwala, Naseem",Mrk1014_a_08_TE,"Maloney, Philip; Kamenetzky, Julia; Spinoglio, Luigi; Schirm, Maximilien; Wilson, Christine; Glenn, Jason; Pereira Santaella, Miguel;",10.917302282371313,0.5369664088675071,0.493685,uid://A001/X121/X27b,uid://A001/X121/X27c,uid://A002/Xa25bbf/X4125,Probing the AGN activity and molecular interstellar medium in ultra-luminous infrared galaxies using CH,S,TARGET,13.591839276478188,1.0213607897630932,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2013.1.00535.S,455.89 .. 471.13
2013.1.00535.S,Mrk1014,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[8],0.22850654107401822,976.5240099979247,12m,,120.96,2016-08-12,"[458.99..460.86GHz,976.52kHz,9.3mJy/beam@10km/s,842.3uJy/beam@native, XX YY] U [460.41..462.29GHz,976.52kHz,9mJy/beam@10km/s,818.2uJy/beam@native, XX YY] U [471.74..473.72GHz,31248.77kHz,18mJy/beam@10km/s,1.6mJy/beam@native, XX YY] U [472.85..474.84GHz,31248.77kHz,33.3mJy/beam@10km/s,3mJy/beam@native, XX YY]",633.2504734763451,XX YY,2015-06-03T11:40:00,"Rangwala, Naseem",Mrk1014_b_08_TE,"Maloney, Philip; Kamenetzky, Julia; Spinoglio, Luigi; Schirm, Maximilien; Wilson, Christine; Glenn, Jason; Pereira Santaella, Miguel;",9.031331037806936,0.5415902866546959,0.53832096,uid://A001/X121/X27f,uid://A001/X121/X280,uid://A002/Xa216e2/X2d6b,Probing the AGN activity and molecular interstellar medium in ultra-luminous infrared galaxies using CH,S,TARGET,13.492914263584487,1.089175149671649,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2013.1.00535.S,458.99 .. 474.84
2012.1.00611.S,pg_0157+001,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[7],0.14732892864972238,976.5029143290341,12m,,1935.36,2016-09-26,"[294.42..296.41GHz,31248.09kHz,684.8uJy/beam@10km/s,48.1uJy/beam@native, XX YY] U [296.43..298.31GHz,976.50kHz,691.6uJy/beam@10km/s,50.3uJy/beam@native, XX YY] U [306.48..308.35GHz,976.50kHz,697.1uJy/beam@10km/s,51.6uJy/beam@native, XX YY] U [308.37..310.35GHz,31248.09kHz,703.1uJy/beam@10km/s,50.5uJy/beam@native, XX YY]",949.3461203518006,XX YY,2015-08-14T07:02:52,"Evans, Aaron",pg0157_297_12m_C32-6,"Barthel, Peter; Frayer, David; Aalto, Susanne; Privon, George; Surace, Jason; Kimball, Amy; Lacy, Mark; Sakamoto, Kazushi; Kim, Dong-Chan;",0.6847646712195564,0.02502833626438749,0.58791834,uid://A002/X684eb5/X16c,uid://A002/X684eb5/X16d,uid://A002/Xa80456/X1ef4,"Dense, Warm Molecular Gas and Star Formation in CO Luminous QSO Hosts",S,TARGET,20.834

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64


...

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2015.1.00027.S,IRAS_15250+3609,231.7476042,35.9770083,57.81138944317473,55.942591714895805,[6],0.7446709201088819,3905.8696707024346,12m,,695.52,2017-04-07,"[251.00..252.87GHz,3905.87kHz,1.2mJy/beam@10km/s,80.5uJy/beam@native, XX YY] U [252.58..254.45GHz,3905.87kHz,1.2mJy/beam@10km/s,80.9uJy/beam@native, XX YY] U [253.94..255.81GHz,3905.87kHz,1.2mJy/beam@10km/s,81.5uJy/beam@native, XX YY]",4576.901809055315,XX YY,2016-03-13T08:07:55,"Imanishi, Masatoshi",IRAS_152_a_06_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.2029148872502589,0.046741056089058,0.7532617,uid://A001/X2fe/X999,uid://A001/X2fe/X99a,uid://A002/Xb07033/Xbee,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,24.861157423179264,3.5203068341378287,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00027.S,"[array([251. , 252.87]), array([252.58, 254.45]), array([253.94, 255.81])]"
2015.1.00027.S,IRAS_15250+3609,231.7476042,35.9770083,57.81138944317473,55.942591714895805,[6],0.7611579362374217,3905.870425599961,12m,,695.52,2017-04-07,"[242.49..244.37GHz,3905.87kHz,1.6mJy/beam@10km/s,104.6uJy/beam@native, XX YY] U [244.19..246.06GHz,3905.87kHz,1.2mJy/beam@10km/s,79.1uJy/beam@native, XX YY] U [256.76..258.64GHz,3905.87kHz,1.2mJy/beam@10km/s,82.5uJy/beam@native, XX YY] U [258.56..260.44GHz,3905.87kHz,1.2mJy/beam@10km/s,83.1uJy/beam@native, XX YY]",4495.658820879416,XX YY,2016-03-13T08:42:16,"Imanishi, Masatoshi",IRAS_152_b_06_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.1984279457096467,0.04293244075819304,0.70880556,uid://A001/X2fe/X9b1,uid://A001/X2fe/X9b2,uid://A002/Xb07033/Xc9b,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,25.053096770376968,3.5982464366985862,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00027.S,"[array([242.49, 244.37]), array([244.19, 246.06]), array([256.76, 258.64]), array([258.56, 260.44])]"
2015.1.00027.S,IRAS_15250+3609,231.74760419996366,35.97700830000605,57.81138944318906,55.942591714924774,[7],0.5224931493852693,3905.9285681411147,12m,,574.56,2017-06-27,"[323.11..324.98GHz,3906.02kHz,12.6mJy/beam@10km/s,955.8uJy/beam@native, XX YY] U [324.73..326.60GHz,3906.02kHz,26.6mJy/beam@10km/s,2mJy/beam@native, XX YY] U [334.97..336.84GHz,3906.02kHz,1.8mJy/beam@10km/s,137.1uJy/beam@native, XX YY] U [336.90..338.78GHz,3906.02kHz,1.7mJy/beam@10km/s,134.4uJy/beam@native, XX YY]",3456.1885897115026,XX YY,2016-03-30T07:43:53,"Imanishi, Masatoshi",IRAS_152_a_07_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.7333811271734179,0.09538763591168624,0.5932493,uid://A001/X2fe/X9c5,uid://A001/X2fe/X9c6,uid://A002/Xb20b6d/X1b43,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,19.036482543338728,2.8909805565304407,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",G

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2011.0.00099.S,am2055-425,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[3],1.5851527436418775,976.4997367331086,12m,,544.32,2013-08-24,"[96.18..98.06GHz,976.50kHz,3.4mJy/beam@10km/s,140.9uJy/beam@native, XX YY] U [97.52..99.39GHz,976.50kHz,3.4mJy/beam@10km/s,141.1uJy/beam@native, XX YY] U [107.88..109.75GHz,976.50kHz,3.5mJy/beam@10km/s,152.4uJy/beam@native, XX YY] U [109.57..111.45GHz,976.50kHz,3.6mJy/beam@10km/s,160.3uJy/beam@native, XX YY]",2626.5675747178216,XX YY,2012-05-05T10:55:19,"Iono, Daisuke",group 7-B3-EXTENDED,"Kaneko, Hiroyuki; Narayanan, Desika; Espada, Daniel; Tamura, Yoichi; Yun, Min; Hatsukade, Bunyo; Kawabe, Ryohei; Crocker, Alison; Ueda, Junko;",3.3716441300933133,0.07401316667831156,1.3472952,,uid://A002/X391d0b/X126,uid://A002/X3f99b1/X3ec,Reformation of Cold Molecular Disks in Merger Remnants,S,TARGET,60.6837112105593,6.78741766678039,Y,4,Merging and interacting galaxies,Galaxy evolution,2011.0.00099.S,"[array([96.18, 98.06]), array([97.52, 99.39]), array([107.88, 109.75]), array([109.57, 111.45])]"
2011.0.00020.S,IRAS 20551-4250,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[7],0.5438087755702041,976.4846833667451,12m,,2419.2,2014-01-16,"[334.61..336.50GHz,976.52kHz,2.2mJy/beam@10km/s,172.3uJy/beam@native, XX YY] U [346.72..348.61GHz,976.52kHz,2.2mJy/beam@10km/s,172.2uJy/beam@native, XX YY]",839.7251946710148,XX YY,2012-06-02T07:23:32,"Imanishi, Masatoshi",IRAS 20551-4250b,"Nakanishi, Kouichiro; Imase, Keisuke; Saitou, Yuriko;",2.190071765663321,0.12182527116334572,0.90417093,,uid://A002/X303d22/X98,uid://A002/X41c27b/X4f5,Molecular line flux ratios and AGN feedback in gas/dust-rich galaxies,S,TARGET,18.442728615724533,2.933106164829256,Y,6,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00020.S,"[array([334.61, 336.5 ]), array([346.72, 348.61])]"
2011.0.00020.S,IRAS 20551-4250,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[7],0.5806443803584678,976.4845932426288,12m,,2419.2,2014-01-16,"[338.93..340.82GHz,976.52kHz,2.7mJy/beam@10km/s,208.9uJy/beam@native, XX YY] U [340.87..342.77GHz,976.52kHz,2.7mJy/beam@10km/s,209.3uJy/beam@native, XX YY] U [350.92..352.82GHz,976.52kHz,2.9mJy/beam@10km/s,228.2uJy/beam@native, XX YY] U [352.62..354.52GHz,976.52kHz,2.9mJy/beam@10km/s,230.3uJy/beam@native, XX YY]",825.7289953119977,XX YY,2012-06-01T10:06:09,"Imanishi, Masatoshi",IRAS 20551-4250a,"Nakanishi, Kouichiro; Imase, Keisuke; Saitou, Yuriko;",2.684324705068637,0.10925105636495278,1.0055575,,uid://A002/X303d22/X96,uid://A002/X41b513/X121,Molecular line flux ratios and AGN feedback in gas/dust-rich galaxies,S,TARGET,18.1707008427369,2.5384246738093554,Y,6,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00020.S,"[array([338.93, 340.82]), array([340.87, 342.77]), array([350.92, 352.82]), array([352.62, 354.52])]"
2013.1.00033.S,IRAS_20551-4250,314.611662,-42.650086,358.6409003766878,-40.79950836350139,[6],0.5304936568641552,976.4730490527244,12m,,786.24,2015-12-17,"[253.99..255.86GHz,976.47kHz,1mJy/beam@10km/s,68.6uJ

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64


Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2011.0.00175.S,NGC6240,253.24541666666667,2.4008333333333334,20.728953939244548,27.290831048964122,[7],0.40699248388678805,31246.46056403259,12m,,8467.2,2013-09-06,"[331.76..333.78GHz,31248.28kHz,2.3mJy/beam@10km/s,169.6uJy/beam@native, XX YY] U [333.62..335.64GHz,31248.28kHz,1.7mJy/beam@10km/s,129.1uJy/beam@native, XX YY] U [343.69..345.71GHz,31248.28kHz,1.7mJy/beam@10km/s,126.1uJy/beam@native, XX YY] U [345.37..347.39GHz,31248.28kHz,1.7mJy/beam@10km/s,127uJy/beam@native, XX YY]",26964.94943158315,XX YY,2012-04-22T09:22:11,"Scoville, Nick",NGC6240 -BAND7,"Narayanan, Desika; Koda, Jin; Walter, Fabian; Hernquist, Lars; Hayward, Christopher; Barnes, Joshua; Manohar, Swarnima; Thompson, Todd; Robertson, Brant; van der Werf, Paul; Brown, Robert; Genzel, Reinhard; Tacconi, Linda; Davies, Richard; Sheth, Kartik; Kartaltepe, Jeyhan;",1.6625390877988098,0.06747232703276126,0.85247904,,uid://A002/X327408/X276,uid://A002/X3ddbb0/X866,Merging IR-Luminous Galaxies -- Arp 220 and NGC 6240,S,TARGET,18.553705276848476,2.1615023071827633,Y,3,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00175.S,"[array([331.76, 333.78]), array([333.62, 335.64]), array([343.69, 345.71]), array([345.37, 347.39])]"
2011.0.00175.S,NGC 6240,253.24541666666667,2.4008333333333334,20.728953939244548,27.290831048964122,[9],0.19768573187620575,31247.609602674416,12m,,6864.48,2013-11-05,"[689.45..691.45GHz,31247.97kHz,14.4mJy/beam@10km/s,1.5mJy/beam@native, XX YY] U [691.00..693.00GHz,31247.97kHz,15.7mJy/beam@10km/s,1.7mJy/beam@native, XX YY] U [693.07..695.07GHz,31247.97kHz,15.5mJy/beam@10km/s,1.7mJy/beam@native, XX YY] U [695.37..697.37GHz,31247.97kHz,15.7mJy/beam@10km/s,1.7mJy/beam@native, XX YY]",13432.348285746839,XX YY,2012-05-20T04:05:00,"Scoville, Nick",NGC6240 - BAND9,"Narayanan, Desika; Koda, Jin; Walter, Fabian; Hernquist, Lars; Hayward, Christopher; Barnes, Joshua; Manohar, Swarnima; Thompson, Todd; Robertson, Brant; van der Werf, Paul; Brown, Robert; Genzel, Reinhard; Tacconi, Linda; Davies, Richard; Sheth, Kartik; Kartaltepe, Jeyhan;",14.379834380929372,0.8227466720362818,0.48359513,,uid://A002/X327408/X279,uid://A002/X40e374/X6cb,Merging IR-Luminous Galaxies -- Arp 220 and NGC 6240,S,TARGET,9.085637654606183,0.972280616470111,Y,3,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00175.S,"[array([689.45, 691.45]), array([691., 693.]), array([693.07, 695.07]), array([695.37, 697.37])]"
2013.1.00049.S,NGC_6240,253.24525416666663,2.400986,20.729013636056607,27.29104750337486,[3],1.7466723039466299,1952.8728015528832,12m,,2842.56,2016-08-07,"[94.71..96.59GHz,1952.88kHz,697.6uJy/beam@10km/s,28.8uJy/beam@native, XX YY] U [106.64..108.52GHz,1952.88kHz,719.2uJy/beam@10km/s,31.5uJy/beam@native, XX YY]",5394.400598303093,XX YY,2015-04-11T06:03:58,"Cicone, Claudia",NGC_6240_a_03_TE,"Papadopoulos, Padelis; Thi, Wing-Fai; Wagg, Jeff; Maiolino, Roberto; Zhang, Zhiyu; Feruglio, Chiara;",0.6976460758163437,0.021235647531750175,7.0027866,uid://A001/X197/X1d,uid://A001/X197/X1e,uid://A002/X9de499/X28d7,An Unpreced

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]"
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64
2013.1.00525.S,Mrk_463,209.01140244190438,18.37174611111111,5.822314923910312,72.74702531629733,[6],0.2272003214754323,15625.028077672943,12m,,3084.48,2016-12-16,"[218.54..220.41GHz,15625.29kHz,857.4uJy/beam@10km/s,53.6uJy/beam@native, XX YY] U [220.06..221.93GHz,15625.29kHz,856.6uJy/beam@10km/s,53.7uJy/beam@native, XX YY] U [232.25..234.11GHz,15625.29kHz,886.4uJy/beam@10km/s,57.1uJy/beam@native, XX YY] U [234.88..236.75GHz,15625.29kHz,975.7uJy/beam@10km/s,63.2uJy/beam@native, XX YY]",19786.139385974217,XX YY,2015-08-16T22:34:32,"Treister, Ezequiel",Mrk_463_a_06_TE,"Koss, Michael; Cales, Sabrina; Comerford, Julia; Sanders, David; Urry, Meg; Muller-Sanchez, Francisco; Schawinski, Kevin; Casey, Caitlin; Nagar, Neil; Evans, Aaron; Bauer, Franz; Messias, Hugo;",0.8566301071350585,0.028259343578282532,1.6854169,uid://A001/X13f/Xc8,uid://A001/X13f/Xc9,uid://A002/Xa830fc/X4abc,Caught in a Cosmic Tango: ALMA Spies the Interplay Between Dual AGN in Merger Remnant Galaxies,S,TARGET,27.67465510621696,1.352748628556253,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Merging and interacting galaxies",Active galaxies,2013.1.00525.S,218.54 .. 236.75


In [12]:
print('galaxies with ALMA data:', n)

galaxies with ALMA data: 32


## 2. Create a clean, formatted dictionary of results

Copy `.queryResults` to a new dictionary `herusALMA` so we can play around with and clean up the results properly

In [13]:
import copy
# copy results to new dictionary
herusALMA = copy.deepcopy(herusQuery.queryResults)

Now we can replace the positional dictionary keys with the target names from the HERUS survey

In [14]:
for targetname, key in zip(HERUS_sample['name'], herusALMA.keys()):
    herusALMA[targetname] = herusALMA.pop(key)

In [15]:
herusALMA.keys()

dict_keys(['IRAS 00397-1312', '3C273', 'IRAS 20100-4156', 'Mrk1014', 'IRAS 03521+0028', 'IRAS 03158+4227', 'IRAS 16090-0139', 'Mrk231', 'IRAS 00188-0856', 'IRAS 07598+6508', 'IRAS 08311-2459', 'IRAS 20087-0308', 'IRAS 10378+1109', 'IRAS 19297-0406', 'IRAS 17208-0014', 'IRAS 14348-1447', 'IRAS 13451+1232', 'IRAS 23230-6926', 'IRAS 12071-0444', 'IRAS 11095-0238', 'IRAS 23253-5415', 'IRAS 01003-2238', 'IRAS 15462-0450', 'IRAS 20414-1651', 'IRAS 09022-3615', 'IRAS 06206-6315', 'IRAS 13120-5453', 'IRAS 22491-1808', 'IRAS 23365+3604', 'IRAS 08572+3915', 'IRAS 06035-7102', 'IRAS 14378-3651', 'Arp220', 'Mrk273', 'IRAS 05189-2524', 'IRAS 10565+2448', 'IRAS 19254-7245', 'IRAS 23128-5919', 'IRAS 15250+3609', 'IRAS 20551-4250', 'UGC5101', 'NGC6240', 'Mrk463'])

Use `my_dict.pop('key', None)` to remove targets with no ALMA data from the `herusALMA` dictionary

In [16]:
for key in list(herusALMA.keys()):
    if len(herusALMA[key])==0: # if no archive data, remove entry
        herusALMA.pop(key, None)
    else:
        display(herusALMA[key])

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2016.1.01073.S,3C273,187.2778539638189,2.052425419153435,289.9507708791571,64.35995216172215,[6],0.6366613102530091,62493.24686378306,12m,,6531.758,2018-03-03,"[222.99..224.96GHz,62493.29kHz,574.4uJy/beam@10km/s,35.1uJy/beam@native, XX XY YY] U [224.99..226.96GHz,62493.29kHz,589.3uJy/beam@10km/s,36.2uJy/beam@native, XX XY YY] U [238.99..240.96GHz,62493.29kHz,586.9uJy/beam@10km/s,37.1uJy/beam@native, XX XY YY] U [240.99..242.96GHz,62493.29kHz,592.6uJy/beam@10km/s,37.6uJy/beam@native, XX XY YY]",77103.74004787063,XX XY YX YY,2016-12-20T09:33:13,"Hovatta, Talvikki",3C273_a_06_TM1,"Savolainen, Tuomas; O'Sullivan, Shane; Tchekhovskoy, Alexander;",0.5743872343921695,0.01823828560997868,1.3283782,uid://A001/X87a/X885,uid://A001/X87a/X886,uid://A002/Xbbf310/X1d61,Probing the magnetic fields at the base of relativistic jets through Faraday rotation observations,S,TARGET,27.04176102625552,3.2923768678479095,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Outflows, jets, feedback",Active galaxies,2016.1.01073.S,"[array([222.99, 224.96]), array([224.99, 226.96]), array([238.99, 240.96]), array([240.99, 242.96])]"
2016.1.01216.V,3C273,187.27785368739794,2.052443798078459,289.95076175698233,64.35997011387919,[3],3.2590125179667355,15624.786288841999,12m,,6096.384,2018-10-21,"[85.33..87.20GHz,15624.96kHz,843.4uJy/beam@10km/s,33uJy/beam@native, XX XY YY] U [87.33..89.20GHz,15624.96kHz,826.2uJy/beam@10km/s,32.7uJy/beam@native, XX XY YY] U [97.39..99.26GHz,15624.96kHz,776.2uJy/beam@10km/s,32.5uJy/beam@native, XX XY YY] U [99.33..101.20GHz,15624.96kHz,772.6uJy/beam@10km/s,32.6uJy/beam@native, XX XY YY]",46285.76402485768,XX XY YX YY,2017-04-04T00:24:39,"Akiyama, Kazunori",vexed 4 April 3C273_a_03_TM1,"Fish, Vincent; Asada, Keiichi; Johnson, Michael; Gomez, Jose L.; Pu, Hung-Yi; Nagai, Hiroshi; Nakamura, Masanori; Honma, Mareki; Hada, Kazuhiro; Blackburn, Lindy; Doeleman, Sheperd; Mizuno, Yosuke; Koyama, Shoko; Chael, Andrew; Lonsdale, Colin; Marscher, Alan; Gabuzda, Denise; Bouman, Katherine;",0.7726293315855453,0.016358689847191233,0.68157107,uid://A001/X11b3/Xa,uid://A001/X11b3/Xd,uid://A002/Xbeb207/X79e,Probing the active collimation region of the relativistic jet in 3C 273,V,APPPHASE_ACTIVE TARGET,67.54820924137793,20.45384637583945,Y,0,"Active Galactic Nuclei (AGN)/Quasars (QSO), Outflows, jets, feedback",Active galaxies,2016.1.01216.V,"[array([85.33, 87.2 ]), array([87.33, 89.2 ]), array([97.39, 99.26]), array([ 99.33, 101.2 ])]"
2018.1.00357.S,3C273,187.27785012674357,2.0524471126437542,289.95075216751826,64.35997264960277,[6],0.26552431459364534,62495.214616261044,12m,,3084.48,2019-12-24,"[223.00..224.97GHz,62495.54kHz,855.5uJy/beam@10km/s,52.3uJy/beam@native, XX XY YY] U [225.00..226.97GHz,62495.54kHz,879uJy/beam@10km/s,54uJy/beam@native, XX XY YY] U [239.00..240.97GHz,62495.54kHz,874.2uJy/beam@10km/s,55.3uJy/beam@native, XX XY YY] U [241.00..242.97GHz,62495.54kHz,883.3uJy/beam@10km/s,56.1uJy/beam@native, XX XY YY]",77106.16784740346,XX XY YX YY,2018-11-11T13:42:10,"Hovatta, Talvikki",3C273_a_06_TM1,"Savolainen, Tuomas; Marti-Vidal, Ivan; O'Sullivan, Shane; Tchekhovskoy

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2013.1.00659.S,IRAS_20100-4156,303.372954,-41.793189,358.763123154025,-32.43952353691461,[3],1.4766248761805358,976.4941902735678,12m,,12670.56,2015-09-27,"[99.20..101.19GHz,31247.92kHz,664.6uJy/beam@10km/s,27.2uJy/beam@native, XX YY] U [101.11..102.99GHz,976.50kHz,665.5uJy/beam@10km/s,28.4uJy/beam@native, XX YY]",2842.375726278533,XX YY,2014-06-09T09:25:29,"Spoon, Henrik",IRAS_201_a_03_TE,"Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Afonso, Jose; Gonzalez-Alfonso, Eduardo; Pearson, Chris; Farrah, Duncan; Bernard-Salas, Jeronimo;",0.6646029848565244,0.019617680035142075,0.7210153,uid://A001/X121/X2b1,uid://A001/X121/X2b2,uid://A002/X83b318/Xcda,Detecting in CO(1-0) the Strongest Molecular Outflow found by Herschel in the Southern Sky,S,TARGET WVR,62.31809481412121,12.646461432597606,Y,2,"Merging and interacting galaxies, Outflows, jets, feedback",Galaxy evolution,2013.1.00659.S,"[array([ 99.2 , 101.19]), array([101.11, 102.99])]"
2016.1.00615.S,IRAS20100-4156,303.37308750002956,-41.7931175000075,358.7632252909276,-32.43961064892275,[6],0.5799155173055671,7813.039344573977,12m,,5201.28,2018-01-28,"[215.86..217.73GHz,7813.05kHz,627uJy/beam@10km/s,38.9uJy/beam@native, XX YY] U [229.65..231.52GHz,7813.05kHz,630.3uJy/beam@10km/s,40.4uJy/beam@native, XX YY]",10117.512051340444,XX YY,2016-11-10T23:16:56,"Gowardhan, Avani",IRAS2010_a_06_TM1,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",0.6270057497181435,0.028027108446736643,1.912678,uid://A001/X87a/X852,uid://A001/X87a/X853,uid://A002/Xba6edb/X5b4f,Probing Dense Gas Physics in the Most Extreme Southern Molecular Outflow,S,TARGET,28.163672820080123,3.0889484522108206,Y,0,"Outflows, jets, feedback, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2016.1.00615.S,"[array([215.86, 217.73]), array([229.65, 231.52])]"
2016.1.00615.S,iras20100-4156,303.3730874999803,-41.7931175000082,358.76322529091993,-32.439610648886585,[7],0.6295592299396062,15623.376506426828,12m,,8799.84,2018-06-23,"[302.56..304.42GHz,15623.38kHz,665.6uJy/beam@10km/s,48.9uJy/beam@native, XX YY] U [312.85..314.71GHz,15623.38kHz,694.3uJy/beam@10km/s,51.9uJy/beam@native, XX YY] U [314.86..315.79GHz,15623.38kHz,717.4uJy/beam@10km/s,76uJy/beam@native, XX YY] U [315.29..316.22GHz,15623.38kHz,764.7uJy/beam@10km/s,81uJy/beam@native, XX YY]",14810.402563980124,XX YY,2017-05-01T07:26:05,"Gowardhan, Avani",iras2010_a_07_TM1,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Rigopoulou, Dimitra; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",0.6655851575481608,0.02994593449700171,0.75314915,uid://A001/X87a/X856,uid://A001/X87a/X857,uid://A002/Xbfd686/X1571,Probing Dense Gas Physics in the Most Extreme Southern Molecular Outflow,S,TARGET,20.362940126102963,3.192691776160348,Y,0,"Outflows, jets, feedback, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2016.1.00615.S,"[array([302.56, 304.42]), array([312.85, 314.71]), array([314.86, 315.79]), array([315.29, 316.22])]"
2018.1.00065.S,iras_20100-4156,303.3731333333849

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]"
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64
2013.1.00535.S,Mrk1014,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[8],0.20453740001916954,7812.167591985681,12m,,120.96,2016-08-10,"[455.89..457.76GHz,7812.17kHz,11.7mJy/beam@10km/s,1.1mJy/beam@native, XX YY] U [456.94..458.81GHz,7812.17kHz,10.9mJy/beam@10km/s,985.3uJy/beam@native, XX YY] U [467.88..469.86GHz,31248.67kHz,11.4mJy/beam@10km/s,1mJy/beam@native, XX YY] U [469.15..471.13GHz,31248.67kHz,15.2mJy/beam@10km/s,1.3mJy/beam@native, XX YY]",5104.331575671292,XX YY,2015-06-06T12:23:46,"Rangwala, Naseem",Mrk1014_a_08_TE,"Maloney, Philip; Kamenetzky, Julia; Spinoglio, Luigi; Schirm, Maximilien; Wilson, Christine; Glenn, Jason; Pereira Santaella, Miguel;",10.917302282371313,0.5369664088675071,0.493685,uid://A001/X121/X27b,uid://A001/X121/X27c,uid://A002/Xa25bbf/X4125,Probing the AGN activity and molecular interstellar medium in ultra-luminous infrared galaxies using CH,S,TARGET,13.591839276478188,1.0213607897630932,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2013.1.00535.S,455.89 .. 471.13
2013.1.00535.S,Mrk1014,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[8],0.22850654107401822,976.5240099979247,12m,,120.96,2016-08-12,"[458.99..460.86GHz,976.52kHz,9.3mJy/beam@10km/s,842.3uJy/beam@native, XX YY] U [460.41..462.29GHz,976.52kHz,9mJy/beam@10km/s,818.2uJy/beam@native, XX YY] U [471.74..473.72GHz,31248.77kHz,18mJy/beam@10km/s,1.6mJy/beam@native, XX YY] U [472.85..474.84GHz,31248.77kHz,33.3mJy/beam@10km/s,3mJy/beam@native, XX YY]",633.2504734763451,XX YY,2015-06-03T11:40:00,"Rangwala, Naseem",Mrk1014_b_08_TE,"Maloney, Philip; Kamenetzky, Julia; Spinoglio, Luigi; Schirm, Maximilien; Wilson, Christine; Glenn, Jason; Pereira Santaella, Miguel;",9.031331037806936,0.5415902866546959,0.53832096,uid://A001/X121/X27f,uid://A001/X121/X280,uid://A002/Xa216e2/X2d6b,Probing the AGN activity and molecular interstellar medium in ultra-luminous infrared galaxies using CH,S,TARGET,13.492914263584487,1.089175149671649,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2013.1.00535.S,458.99 .. 474.84
2012.1.00611.S,pg_0157+001,29.9593917,0.3947228,156.568111665194,-57.938940388216686,[7],0.14732892864972238,976.5029143290341,12m,,1935.36,2016-09-26,"[294.42..296.41GHz,31248.09kHz,684.8uJy/beam@10km/s,48.1uJy/beam@native, XX YY] U [296.43..298.31GHz,976.50kHz,691.6uJy/beam@10km/s,50.3uJy/beam@native, XX YY] U [306.48..308.35GHz,976.50kHz,697.1uJy/beam@10km/s,51.6uJy/beam@native, XX YY] U [308.37..310.35GHz,31248.09kHz,703.1uJy/beam@10km/s,50.5uJy/beam@native, XX YY]",949.3461203518006,XX YY,2015-08-14T07:02:52,"Evans, Aaron",pg0157_297_12m_C32-6,"Barthel, Peter; Frayer, David; Aalto, Susanne; Privon, George; Surace, Jason; Kimball, Amy; Lacy, Mark; Sakamoto, Kazushi; Kim, Dong-Chan;",0.6847646712195564,0.02502833626438749,0.58791834,uid://A002/X684eb5/X16c,uid://A002/X684eb5/X16d,uid://A002/Xa80456/X1ef4,"Dense, Warm Molecular Gas and Star Formation in CO Luminous QSO Hosts",S,TARGET,20.834

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2017.1.00057.S,IRAS_16090-0139,242.91845833332428,-1.7850666666700028,10.247310138111743,33.84096830966344,[6],0.128983080522839,1952.9525474911736,12m,,483.84,2019-02-15,"[233.60..235.48GHz,1952.95kHz,1.4mJy/beam@10km/s,90.9uJy/beam@native, XX YY] U [235.08..236.96GHz,1952.95kHz,1.4mJy/beam@10km/s,90.7uJy/beam@native, XX YY] U [236.59..238.46GHz,1952.95kHz,1.4mJy/beam@10km/s,90.5uJy/beam@native, XX YY]",2454.9926609028557,XX YY,2017-12-16T14:40:46,"Imanishi, Masatoshi",IRAS_160_a_06_TM1,"Izumi, Takuma; Nakanishi, Kouichiro; Kikuta, Satoshi;",1.3915856200929795,0.05235753784181108,1.9102613,uid://A001/X1284/X13e8,uid://A001/X1284/X13e9,uid://A002/Xc7fa6f/X3bb5,ALMA survey of extremely deeply buried AGN in ultraluminous infrared galaxies,S,TARGET,26.69111836447081,1.1860718547706417,Y,0,"Galactic centres/nuclei, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2017.1.00057.S,"[array([233.6 , 235.48]), array([235.08, 236.96]), array([236.59, 238.46])]"
2018.1.00503.S,IRAS_16090-0139,242.9184583333361,-1.785066666670016,10.247310138119815,33.84096830965373,[3],11.260375954215514,15625.329118691894,7m,,544.32,2019-10-23,"[99.00..100.98GHz,31250.66kHz,15.5mJy/beam@10km/s,631.2uJy/beam@native, XX YY] U [100.69..102.68GHz,15625.33kHz,16mJy/beam@10km/s,657.4uJy/beam@native, XX YY]",45622.41062585483,XX YY,2018-10-04T19:45:00,"Gowardhan, Avani",IRAS_160_a_03_7M,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Feruglio, Chiara; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",15.45560190459261,0.45528981515242245,5.186,uid://A001/X133f/Xc4,uid://A001/X133f/Xc5,uid://A002/Xd2d9a0/X8b8e,The galaxy-wide impact of powerful feedback in ULIRGs,S,TARGET,62.476894396914744,41.54379966698691,Y,0,Active Galactic Nuclei (AGN)/Quasars (QSO),Active galaxies,2018.1.00503.S,"[array([ 99. , 100.98]), array([100.69, 102.68])]"
2018.1.00503.S,IRAS_16090-0139,242.918738608695,-1.78492807583645,10.247639519304926,33.84081718199658,[7],4.069576134620171,15624.797556225529,7m,,604.8,2019-11-09,"[304.04..306.03GHz,15624.80kHz,13.5mJy/beam@10km/s,960.8uJy/beam@native, XX YY]",15306.201980852387,XX YY,2018-10-30T15:44:02,"Gowardhan, Avani",IRAS_160_a_07_7M,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Feruglio, Chiara; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",13.470774069135333,0.9608161259723592,0.91,uid://A001/X133f/Xc8,uid://A001/X133f/Xc9,uid://A002/Xd42ec5/X7bd3,The galaxy-wide impact of powerful feedback in ULIRGs,S,TARGET,20.653534686519638,15.014221683089215,Y,0,Active Galactic Nuclei (AGN)/Quasars (QSO),Active galaxies,2018.1.00503.S,"[array([304.04, 306.03])]"


Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2017.1.00057.S,IRAS_00188-0856,5.360541666714907,-8.657263888893082,100.52817663669737,-70.21321062746263,[6],0.16414636143315822,1953.3376272747648,12m,,2147.04,2019-02-27,"[234.78..236.65GHz,1953.34kHz,820.1uJy/beam@10km/s,53.1uJy/beam@native, XX YY] U [236.26..238.14GHz,1953.34kHz,714.1uJy/beam@10km/s,46.4uJy/beam@native, XX YY] U [237.78..239.65GHz,1953.34kHz,624.3uJy/beam@10km/s,40.7uJy/beam@native, XX YY]",2443.772557967229,XX YY,2017-12-17T00:42:39,"Imanishi, Masatoshi",IRAS_001_a_06_TM1,"Izumi, Takuma; Nakanishi, Kouichiro; Kikuta, Satoshi;",0.6243035693369126,0.026505505631073924,2.1026454,uid://A001/X1284/X13c4,uid://A001/X1284/X13c5,uid://A002/Xc805c1/X74a,ALMA survey of extremely deeply buried AGN in ultraluminous infrared galaxies,S,TARGET,26.557965118793142,1.3780481331341825,Y,0,"Galactic centres/nuclei, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Active galaxies,2017.1.00057.S,"[array([234.78, 236.65]), array([236.26, 238.14]), array([237.78, 239.65])]"
2018.1.00503.S,IRAS_00188-0856,5.360541666619002,-8.65726388889439,100.52817663643293,-70.21321062743135,[3],11.848804160649292,7812.714009633086,7m,,2328.48,2019-10-30,"[99.50..101.48GHz,31250.86kHz,7.4mJy/beam@10km/s,303.2uJy/beam@native, XX YY] U [101.19..103.19GHz,7812.71kHz,7.4mJy/beam@10km/s,306.9uJy/beam@native, XX YY]",22698.52547069554,XX YY,2018-10-05T01:41:23,"Gowardhan, Avani",IRAS_001_a_03_7M,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Feruglio, Chiara; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",7.4072406087182445,0.21570561577677,5.186,uid://A001/X133f/X98,uid://A001/X133f/X99,uid://A002/Xd2d9a0/Xa32e,The galaxy-wide impact of powerful feedback in ULIRGs,S,TARGET,62.16507519970266,43.71473460076572,Y,0,Active Galactic Nuclei (AGN)/Quasars (QSO),Active galaxies,2018.1.00503.S,"[array([ 99.5 , 101.48]), array([101.19, 103.19])]"
2018.1.00503.S,IRAS_00188-0856,5.360541666617061,-8.657263888894944,100.52817663642702,-70.2132106274312,[7],4.170495085632182,15626.199267457021,7m,,1239.84,2019-11-12,"[292.08..294.07GHz,15626.20kHz,9.7mJy/beam@10km/s,677.7uJy/beam@native, XX YY] U [305.56..307.56GHz,15626.20kHz,9.3mJy/beam@10km/s,665.5uJy/beam@native, XX YY]",15232.929440562155,XX YY,2018-11-06T23:11:43,"Gowardhan, Avani",IRAS_001_a_07_7M,"Spoon, Henrik; Fischer, Jacqueline; Riechers, Dominik; Feruglio, Chiara; Gonzalez-Alfonso, Eduardo; Farrah, Duncan;",9.307571438192607,0.47483256716741695,0.91,uid://A001/X133f/X9c,uid://A001/X133f/X9d,uid://A002/Xd4b9e4/Xb694,The galaxy-wide impact of powerful feedback in ULIRGs,S,TARGET,21.012716138790957,15.386550262871532,Y,0,Active Galactic Nuclei (AGN)/Quasars (QSO),Active galaxies,2018.1.00503.S,"[array([292.08, 294.07]), array([305.56, 307.56])]"


...

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]"
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64
2015.1.00102.S,ESO_148-IG002,348.94512640068314,-59.05440002118826,323.72492397354347,-54.04149587668204,[7],0.30535732665205584,7812.208871370951,12m,mosaic,1386.162,2017-11-08,"[326.83..328.71GHz,7812.45kHz,3.9mJy/beam@10km/s,296.7uJy/beam@native, XX YY] U [328.53..330.51GHz,31249.80kHz,2.4mJy/beam@10km/s,176.3uJy/beam@native, XX YY] U [338.63..340.50GHz,7812.45kHz,1.7mJy/beam@10km/s,130.7uJy/beam@native, XX YY] U [340.39..342.26GHz,7812.45kHz,1.7mJy/beam@10km/s,130.8uJy/beam@native, XX YY]",6842.63715221715,XX YY,2016-06-04T09:01:00,"Iono, Daisuke",ESO_148-_b_07_TE,"Yun, Min; Ando, Misaki; Michiyama, Tomonari; Saito, Toshiki; Nakanishi, Kouichiro; Ueda, Junko; Bouché, Nicolas; Motohara, Kentaro; Yamashita, Takuji; Inami, Hanae;",1.6781011374798873,0.07891194491986142,0.69148076,uid://A001/X5a3/X131,uid://A001/X5a3/X132,uid://A002/Xb3e648/X3383,Warm and Dense Molecular Gas in Local Merging ULIRGs,S,TARGET,27.43144760642458,3.203728469750336,Y,1,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00102.S,326.83 .. 342.26
2015.1.00102.S,ESO_148-IG002,348.9451264006825,-59.054400021190105,323.724923973542,-54.0414958766804,[7],0.2187160383178488,15624.511189987385,12m,mosaic,1557.047,2017-11-25,"[315.93..317.79GHz,15625.28kHz,2mJy/beam@10km/s,153.9uJy/beam@native, XX YY] U [317.64..319.62GHz,31250.56kHz,2.3mJy/beam@10km/s,164uJy/beam@native, XX YY] U [327.74..329.61GHz,15625.28kHz,3.7mJy/beam@10km/s,283.8uJy/beam@native, XX YY] U [329.70..331.56GHz,15625.28kHz,2.6mJy/beam@10km/s,196.9uJy/beam@native, XX YY]",14126.92028294014,XX YY,2016-06-15T07:59:49,"Iono, Daisuke",ESO_148-_a_07_TE,"Yun, Min; Ando, Misaki; Michiyama, Tomonari; Saito, Toshiki; Nakanishi, Kouichiro; Ueda, Junko; Bouché, Nicolas; Motohara, Kentaro; Yamashita, Takuji; Inami, Hanae;",2.049418015370391,0.09221004227949108,1.0136164,uid://A001/X2ca/X35,uid://A001/X2ca/X36,uid://A002/Xb44b49/X8470,Warm and Dense Molecular Gas in Local Merging ULIRGs,S,TARGET,28.059809840375166,3.171329104711127,Y,1,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00102.S,315.93 .. 331.56
2016.1.00140.S,ESO148-ig002,348.9448833299702,-59.054427780003536,323.7250628355581,-54.04139719528,[3],0.6268153059835923,3906.5387012137435,12m,,151.2,2017-12-23,"[95.42..97.41GHz,31252.31kHz,2.4mJy/beam@10km/s,96.6uJy/beam@native, XX YY] U [97.38..99.37GHz,31252.31kHz,2.4mJy/beam@10km/s,96.3uJy/beam@native, XX YY] U [107.42..109.41GHz,31252.31kHz,2.5mJy/beam@10km/s,104.2uJy/beam@native, XX YY] U [109.37..111.25GHz,3906.54kHz,2.6mJy/beam@10km/s,113.7uJy/beam@native, XX YY]",10528.313599235355,XX YY,2016-10-03T01:50:59,"Iono, Daisuke",ESO148-i_a_03_TM1,"Yun, Min; Ando, Misaki; Michiyama, Tomonari; Saito, Toshiki; Nakanishi, Kouichiro; Ueda, Junko; Bouché, Nicolas; Motohara, Kentaro; Yamashita, Takuji; Inami, Hanae;",2.377156687692308,0.05100220202994324,1.7431756,uid://A001/X879/X79f,uid://A001/X879/X7a0,uid://A002/Xb8f857/X401,Molecular Gas in Local Merging ULIRGs,S,TARGET,60.9669351

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2015.1.00027.S,IRAS_15250+3609,231.7476042,35.9770083,57.81138944317473,55.942591714895805,[6],0.7446709201088819,3905.8696707024346,12m,,695.52,2017-04-07,"[251.00..252.87GHz,3905.87kHz,1.2mJy/beam@10km/s,80.5uJy/beam@native, XX YY] U [252.58..254.45GHz,3905.87kHz,1.2mJy/beam@10km/s,80.9uJy/beam@native, XX YY] U [253.94..255.81GHz,3905.87kHz,1.2mJy/beam@10km/s,81.5uJy/beam@native, XX YY]",4576.901809055315,XX YY,2016-03-13T08:07:55,"Imanishi, Masatoshi",IRAS_152_a_06_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.2029148872502589,0.046741056089058,0.7532617,uid://A001/X2fe/X999,uid://A001/X2fe/X99a,uid://A002/Xb07033/Xbee,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,24.861157423179264,3.5203068341378287,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00027.S,"[array([251. , 252.87]), array([252.58, 254.45]), array([253.94, 255.81])]"
2015.1.00027.S,IRAS_15250+3609,231.7476042,35.9770083,57.81138944317473,55.942591714895805,[6],0.7611579362374217,3905.870425599961,12m,,695.52,2017-04-07,"[242.49..244.37GHz,3905.87kHz,1.6mJy/beam@10km/s,104.6uJy/beam@native, XX YY] U [244.19..246.06GHz,3905.87kHz,1.2mJy/beam@10km/s,79.1uJy/beam@native, XX YY] U [256.76..258.64GHz,3905.87kHz,1.2mJy/beam@10km/s,82.5uJy/beam@native, XX YY] U [258.56..260.44GHz,3905.87kHz,1.2mJy/beam@10km/s,83.1uJy/beam@native, XX YY]",4495.658820879416,XX YY,2016-03-13T08:42:16,"Imanishi, Masatoshi",IRAS_152_b_06_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.1984279457096467,0.04293244075819304,0.70880556,uid://A001/X2fe/X9b1,uid://A001/X2fe/X9b2,uid://A002/Xb07033/Xc9b,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,25.053096770376968,3.5982464366985862,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00027.S,"[array([242.49, 244.37]), array([244.19, 246.06]), array([256.76, 258.64]), array([258.56, 260.44])]"
2015.1.00027.S,IRAS_15250+3609,231.74760419996366,35.97700830000605,57.81138944318906,55.942591714924774,[7],0.5224931493852693,3905.9285681411147,12m,,574.56,2017-06-27,"[323.11..324.98GHz,3906.02kHz,12.6mJy/beam@10km/s,955.8uJy/beam@native, XX YY] U [324.73..326.60GHz,3906.02kHz,26.6mJy/beam@10km/s,2mJy/beam@native, XX YY] U [334.97..336.84GHz,3906.02kHz,1.8mJy/beam@10km/s,137.1uJy/beam@native, XX YY] U [336.90..338.78GHz,3906.02kHz,1.7mJy/beam@10km/s,134.4uJy/beam@native, XX YY]",3456.1885897115026,XX YY,2016-03-30T07:43:53,"Imanishi, Masatoshi",IRAS_152_a_07_TE,"Nakanishi, Kouichiro; Kikuta, Satoshi; Saitou, Yuriko;",1.7333811271734179,0.09538763591168624,0.5932493,uid://A001/X2fe/X9c5,uid://A001/X2fe/X9c6,uid://A002/Xb20b6d/X1b43,AGN feedback and molecular line flux ratios in dust/gas-rich ultraluminous luminous infrared galaxies,S,TARGET,19.036482543338728,2.8909805565304407,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",G

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2011.0.00099.S,am2055-425,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[3],1.5851527436418775,976.4997367331086,12m,,544.32,2013-08-24,"[96.18..98.06GHz,976.50kHz,3.4mJy/beam@10km/s,140.9uJy/beam@native, XX YY] U [97.52..99.39GHz,976.50kHz,3.4mJy/beam@10km/s,141.1uJy/beam@native, XX YY] U [107.88..109.75GHz,976.50kHz,3.5mJy/beam@10km/s,152.4uJy/beam@native, XX YY] U [109.57..111.45GHz,976.50kHz,3.6mJy/beam@10km/s,160.3uJy/beam@native, XX YY]",2626.5675747178216,XX YY,2012-05-05T10:55:19,"Iono, Daisuke",group 7-B3-EXTENDED,"Kaneko, Hiroyuki; Narayanan, Desika; Espada, Daniel; Tamura, Yoichi; Yun, Min; Hatsukade, Bunyo; Kawabe, Ryohei; Crocker, Alison; Ueda, Junko;",3.3716441300933133,0.07401316667831156,1.3472952,,uid://A002/X391d0b/X126,uid://A002/X3f99b1/X3ec,Reformation of Cold Molecular Disks in Merger Remnants,S,TARGET,60.6837112105593,6.78741766678039,Y,4,Merging and interacting galaxies,Galaxy evolution,2011.0.00099.S,"[array([96.18, 98.06]), array([97.52, 99.39]), array([107.88, 109.75]), array([109.57, 111.45])]"
2011.0.00020.S,IRAS 20551-4250,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[7],0.5438087755702041,976.4846833667451,12m,,2419.2,2014-01-16,"[334.61..336.50GHz,976.52kHz,2.2mJy/beam@10km/s,172.3uJy/beam@native, XX YY] U [346.72..348.61GHz,976.52kHz,2.2mJy/beam@10km/s,172.2uJy/beam@native, XX YY]",839.7251946710148,XX YY,2012-06-02T07:23:32,"Imanishi, Masatoshi",IRAS 20551-4250b,"Nakanishi, Kouichiro; Imase, Keisuke; Saitou, Yuriko;",2.190071765663321,0.12182527116334572,0.90417093,,uid://A002/X303d22/X98,uid://A002/X41c27b/X4f5,Molecular line flux ratios and AGN feedback in gas/dust-rich galaxies,S,TARGET,18.442728615724533,2.933106164829256,Y,6,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00020.S,"[array([334.61, 336.5 ]), array([346.72, 348.61])]"
2011.0.00020.S,IRAS 20551-4250,314.611663,-42.650086,358.64090039986894,-40.79950909879711,[7],0.5806443803584678,976.4845932426288,12m,,2419.2,2014-01-16,"[338.93..340.82GHz,976.52kHz,2.7mJy/beam@10km/s,208.9uJy/beam@native, XX YY] U [340.87..342.77GHz,976.52kHz,2.7mJy/beam@10km/s,209.3uJy/beam@native, XX YY] U [350.92..352.82GHz,976.52kHz,2.9mJy/beam@10km/s,228.2uJy/beam@native, XX YY] U [352.62..354.52GHz,976.52kHz,2.9mJy/beam@10km/s,230.3uJy/beam@native, XX YY]",825.7289953119977,XX YY,2012-06-01T10:06:09,"Imanishi, Masatoshi",IRAS 20551-4250a,"Nakanishi, Kouichiro; Imase, Keisuke; Saitou, Yuriko;",2.684324705068637,0.10925105636495278,1.0055575,,uid://A002/X303d22/X96,uid://A002/X41b513/X121,Molecular line flux ratios and AGN feedback in gas/dust-rich galaxies,S,TARGET,18.1707008427369,2.5384246738093554,Y,6,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00020.S,"[array([338.93, 340.82]), array([340.87, 342.77]), array([350.92, 352.82]), array([352.62, 354.52])]"
2013.1.00033.S,IRAS_20551-4250,314.611662,-42.650086,358.6409003766878,-40.79950836350139,[6],0.5304936568641552,976.4730490527244,12m,,786.24,2015-12-17,"[253.99..255.86GHz,976.47kHz,1mJy/beam@10km/s,68.6uJ

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object
2011.0.00175.S,NGC6240,253.24541666666667,2.4008333333333334,20.728953939244548,27.290831048964122,[7],0.40699248388678805,31246.46056403259,12m,,8467.2,2013-09-06,"[331.76..333.78GHz,31248.28kHz,2.3mJy/beam@10km/s,169.6uJy/beam@native, XX YY] U [333.62..335.64GHz,31248.28kHz,1.7mJy/beam@10km/s,129.1uJy/beam@native, XX YY] U [343.69..345.71GHz,31248.28kHz,1.7mJy/beam@10km/s,126.1uJy/beam@native, XX YY] U [345.37..347.39GHz,31248.28kHz,1.7mJy/beam@10km/s,127uJy/beam@native, XX YY]",26964.94943158315,XX YY,2012-04-22T09:22:11,"Scoville, Nick",NGC6240 -BAND7,"Narayanan, Desika; Koda, Jin; Walter, Fabian; Hernquist, Lars; Hayward, Christopher; Barnes, Joshua; Manohar, Swarnima; Thompson, Todd; Robertson, Brant; van der Werf, Paul; Brown, Robert; Genzel, Reinhard; Tacconi, Linda; Davies, Richard; Sheth, Kartik; Kartaltepe, Jeyhan;",1.6625390877988098,0.06747232703276126,0.85247904,,uid://A002/X327408/X276,uid://A002/X3ddbb0/X866,Merging IR-Luminous Galaxies -- Arp 220 and NGC 6240,S,TARGET,18.553705276848476,2.1615023071827633,Y,3,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00175.S,"[array([331.76, 333.78]), array([333.62, 335.64]), array([343.69, 345.71]), array([345.37, 347.39])]"
2011.0.00175.S,NGC 6240,253.24541666666667,2.4008333333333334,20.728953939244548,27.290831048964122,[9],0.19768573187620575,31247.609602674416,12m,,6864.48,2013-11-05,"[689.45..691.45GHz,31247.97kHz,14.4mJy/beam@10km/s,1.5mJy/beam@native, XX YY] U [691.00..693.00GHz,31247.97kHz,15.7mJy/beam@10km/s,1.7mJy/beam@native, XX YY] U [693.07..695.07GHz,31247.97kHz,15.5mJy/beam@10km/s,1.7mJy/beam@native, XX YY] U [695.37..697.37GHz,31247.97kHz,15.7mJy/beam@10km/s,1.7mJy/beam@native, XX YY]",13432.348285746839,XX YY,2012-05-20T04:05:00,"Scoville, Nick",NGC6240 - BAND9,"Narayanan, Desika; Koda, Jin; Walter, Fabian; Hernquist, Lars; Hayward, Christopher; Barnes, Joshua; Manohar, Swarnima; Thompson, Todd; Robertson, Brant; van der Werf, Paul; Brown, Robert; Genzel, Reinhard; Tacconi, Linda; Davies, Richard; Sheth, Kartik; Kartaltepe, Jeyhan;",14.379834380929372,0.8227466720362818,0.48359513,,uid://A002/X327408/X279,uid://A002/X40e374/X6cb,Merging IR-Luminous Galaxies -- Arp 220 and NGC 6240,S,TARGET,9.085637654606183,0.972280616470111,Y,3,"Starbursts, star formation, Active Galactic Nuclei (AGN)/Quasars (QSO)",Active galaxies,2011.0.00175.S,"[array([689.45, 691.45]), array([691., 693.]), array([693.07, 695.07]), array([695.37, 697.37])]"
2013.1.00049.S,NGC_6240,253.24525416666663,2.400986,20.729013636056607,27.29104750337486,[3],1.7466723039466299,1952.8728015528832,12m,,2842.56,2016-08-07,"[94.71..96.59GHz,1952.88kHz,697.6uJy/beam@10km/s,28.8uJy/beam@native, XX YY] U [106.64..108.52GHz,1952.88kHz,719.2uJy/beam@10km/s,31.5uJy/beam@native, XX YY]",5394.400598303093,XX YY,2015-04-11T06:03:58,"Cicone, Claudia",NGC_6240_a_03_TE,"Papadopoulos, Padelis; Thi, Wing-Fai; Wagg, Jeff; Maiolino, Roberto; Zhang, Zhiyu; Feruglio, Chiara;",0.6976460758163437,0.021235647531750175,7.0027866,uid://A001/X197/X1d,uid://A001/X197/X1e,uid://A002/X9de499/X28d7,An Unpreced

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]"
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64
2013.1.00525.S,Mrk_463,209.01140244190438,18.37174611111111,5.822314923910312,72.74702531629733,[6],0.2272003214754323,15625.028077672943,12m,,3084.48,2016-12-16,"[218.54..220.41GHz,15625.29kHz,857.4uJy/beam@10km/s,53.6uJy/beam@native, XX YY] U [220.06..221.93GHz,15625.29kHz,856.6uJy/beam@10km/s,53.7uJy/beam@native, XX YY] U [232.25..234.11GHz,15625.29kHz,886.4uJy/beam@10km/s,57.1uJy/beam@native, XX YY] U [234.88..236.75GHz,15625.29kHz,975.7uJy/beam@10km/s,63.2uJy/beam@native, XX YY]",19786.139385974217,XX YY,2015-08-16T22:34:32,"Treister, Ezequiel",Mrk_463_a_06_TE,"Koss, Michael; Cales, Sabrina; Comerford, Julia; Sanders, David; Urry, Meg; Muller-Sanchez, Francisco; Schawinski, Kevin; Casey, Caitlin; Nagar, Neil; Evans, Aaron; Bauer, Franz; Messias, Hugo;",0.8566301071350585,0.028259343578282532,1.6854169,uid://A001/X13f/Xc8,uid://A001/X13f/Xc9,uid://A002/Xa830fc/X4abc,Caught in a Cosmic Tango: ALMA Spies the Interplay Between Dual AGN in Merger Remnant Galaxies,S,TARGET,27.67465510621696,1.352748628556253,Y,1,"Active Galactic Nuclei (AGN)/Quasars (QSO), Merging and interacting galaxies",Active galaxies,2013.1.00525.S,218.54 .. 236.75


## 3. Find lines of interest

`herusALMA` is now a clean, readable dictionary containing every target with archiveal ALMA data. `herusALMA` keys are the source names and the values are ALMA archive astropy tables. We now need to check whether the emmission lines we're interested in are observed.

Search our targets for lines we're interested in:
1. Define the line rest frequencies
2. Iteratively redshift the lines and check if frequency is covered by the spectral set up
3. Add boolean line observed column to each target table in `herusALMA` dictonary



In [17]:
def lineObserved(target_frequency, observed_frequency_ranges, linename=""):
    """Loop through the observed spectral windows to check if 
        target_frequency is covered by spectral setup"""
    
    # Initialize boolean line observed flag array (i.e., True = line frequency in archive spw coverage)
    lineObserved = []
    
    # loop over spectral window frequencies for each observation
    for spw_frequencies in observed_frequency_ranges:
        # loop over each spectral window
        for spw in spw_frequencies:
            # if observed frequency is in spw, flag as True and break inner loop (move on to next observation) 
            if spw[0] <= target_frequency <= spw[-1]:
                print(linename,"observed frequency", target_frequency, "GHz",
                          "in range", spw[0], "-", spw[-1])
                lineflag = True # line IS observed
                break
            else:
                lineflag = False # line NOT observed
        
        # Boolean line observed flag for each observation 
        lineObserved.append(lineflag)
        
    return lineObserved
    

### Potential improvements to line finding

1. Getting the rest (and observed) frequencies could be done using splatalogue in `astroquery`
2. Finding the lines is done in the notebook, would be cool to have this as a class method


In [18]:
def observedFreq(restFreq, z):
    """calculate observed frequency based on redshift, z"""
    return restFreq/(1+z)

Search for the following lines:

1. C$^{18}$O ($J=1-0$) @ 110.20135430 GHz
2. $^{13}$CO ($J=1-0$) @ 109.78217340 GHz


In [19]:
# rest frame frequencies
rf_13CO_10 = 110.20135430 # 13CO J=1-0
rf_C18O_10 = 109.78217340 # C18O J=1-0

rest_frequencies = [rf_13CO_10, rf_C18O_10]
line_names = ['13CO J=1-0', 'C18O J=1-0'] # column names for line observed flags 

In [20]:
# loop over targets
for target in herusALMA.keys():
    
    # target redshift
    z = HERUS_sample['z'][HERUS_sample['name']==target][0]

    # observed frequencies
    observed_frequencies = [observedFreq(rf, z) for rf in rest_frequencies]

    # the observed frequency ranges for the target galaxy
    observed_frequency_ranges = herusALMA[target]['Frequency ranges']

    print(target)
    # loop over the target lines, return a boolean flag array and add it to astropy table
    for observed_frequency, linename in zip(observed_frequencies,line_names):
        herusALMA[target][linename]=lineObserved(target_frequency=observed_frequency, observed_frequency_ranges=observed_frequency_ranges, linename=linename)

3C273
IRAS 20100-4156
Mrk1014
IRAS 16090-0139
IRAS 00188-0856
IRAS 20087-0308
IRAS 10378+1109
IRAS 19297-0406
IRAS 17208-0014
13CO J=1-0 observed frequency 105.65805766606528 GHz in range 104.34 - 106.22
C18O J=1-0 observed frequency 105.25615843364619 GHz in range 104.34 - 106.22
IRAS 14348-1447
IRAS 13451+1232
IRAS 23230-6926
IRAS 12071-0444
IRAS 11095-0238
IRAS 01003-2238
IRAS 15462-0450
IRAS 20414-1651
IRAS 09022-3615
IRAS 13120-5453
13CO J=1-0 observed frequency 106.88783157531871 GHz in range 106.36 - 108.24
13CO J=1-0 observed frequency 106.88783157531871 GHz in range 105.97 - 107.84
C18O J=1-0 observed frequency 106.4812545625098 GHz in range 104.76 - 106.63
C18O J=1-0 observed frequency 106.4812545625098 GHz in range 105.97 - 107.84
IRAS 22491-1808
IRAS 08572+3915
IRAS 06035-7102
IRAS 14378-3651
Arp220
13CO J=1-0 observed frequency 108.25280391343891 GHz in range 106.56 - 108.44
13CO J=1-0 observed frequency 108.25280391343891 GHz in range 106.56 - 108.44
13CO J=1-0 observed f

## 4. As an excercise, compare this output with the `runQueryWithLines` result

** DEVELOPMENT ONLY **
We need to reload the module to account for edits to code

In [24]:
from importlib import reload
import ALMAxmatch
reload(ALMAxmatch)



<module 'ALMAxmatch' from '/Users/thbrown/ALMA/think_tank_code/tools/archiveDev/ALMAxmatch/__init__.py'>

We initialize the survey, like in any other case, by feeding it our targets.

In [25]:
# search using SkyCoord array
herusLineQuery = ALMAxmatch.archiveSearch(herusCoords)

display the target dictionary

In [26]:
display(herusLineQuery.targets)

{'coord=(10.5647 deg -12.93412 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (10.5647, -12.93412)>, '20arcsec'),
 'coord=(187.27792 deg 2.05239 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (187.27792, 2.05239)>, '20arcsec'),
 'coord=(303.37308 deg -41.79303 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (303.37308, -41.79303)>, '20arcsec'),
 'coord=(29.95921 deg 0.39462 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (29.95921, 0.39462)>, '20arcsec'),
 'coord=(58.6758 deg 0.61761 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (58.6758, 0.61761)>, '20arcsec'),
 'coord=(49.80167 deg 42.64111 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (49.80167, 42.64111)>, '20arcsec'),
 'coord=(242.91847 deg -1.78516 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (242.91847, -1.78516)>, '20arcsec'),
 'coord=(194.05931 deg 56.87368 deg) radius=20arcsec': (<SkyCoord (ICRS): (r

Run the query, pulling only science data where lines are observed

In [27]:
herusLineQuery.runQueriesWithLines(restFreqs=rest_frequencies, science=True)

display the results

In [28]:
for targetname in herusLineQuery.targets.keys():
    if len(herusLineQuery.queryResults[targetname])>1:
        display(herusLineQuery.queryResults[targetname])

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]",ALMA sanitized source name,NED source name,NED RA,NED Dec,Redshift,Line0 observed,Line1 observed
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64,str14,bytes30,float64,float64,float64,bool,bool
2013.1.00379.S,IRAS_13120-5453,198.77642458333332,-55.156354722222225,306.33810137119167,7.558740947322939,[3],0.8320832838686458,1953.295462665399,12m,,3024.0,2016-09-11,"[93.06..94.94GHz,1953.30kHz,561.7uJy/beam@10km/s,23uJy/beam@native, XX YY] U [94.67..96.55GHz,1953.30kHz,553uJy/beam@10km/s,22.8uJy/beam@native, XX YY] U [104.76..106.63GHz,1953.30kHz,553.1uJy/beam@10km/s,24uJy/beam@native, XX YY] U [106.36..108.24GHz,1953.30kHz,560.4uJy/beam@10km/s,24.5uJy/beam@native, XX YY]",5410.569269720324,XX YY,2015-07-22T21:35:34,"Sliwa, Kazimierz",IRAS_131_b_03_TE,"Wilson, Christine;",0.5529899194942456,0.011765087759369247,3.1714032,uid://A001/X145/X185,uid://A001/X145/X186,uid://A002/Xa66f35/X144,The Forgotten Local Ultra Luminous Infrared Galaxy: IRAS 13120-5453,S,TARGET,62.59205952663193,4.67502438693053,Y,2,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2013.1.00379.S,93.06 .. 108.24,IRAS13120-5453,WKK 2031,198.77649,-55.15645,0.030761,True,True
2015.1.00287.S,IRAS_13120-5453,198.77642458328958,-55.156354722231406,306.33810137116575,7.558740947316102,[3],0.6304682493136776,3906.551321668138,12m,,2782.08,2018-03-20,"[94.08..95.95GHz,3906.55kHz,615.1uJy/beam@10km/s,25.3uJy/beam@native, XX YY] U [95.77..97.65GHz,3906.55kHz,610.4uJy/beam@10km/s,25.3uJy/beam@native, XX YY] U [105.97..107.84GHz,3906.55kHz,608.5uJy/beam@10km/s,26.5uJy/beam@native, XX YY] U [107.66..109.53GHz,3906.55kHz,620.9uJy/beam@10km/s,27.3uJy/beam@native, XX YY]",10692.913267880172,XX YY,2016-09-07T16:54:32,"Sliwa, Kazimierz",IRAS_131_d_03_TE,"Wilson, Christine;",0.6085407594125293,0.013034173471956222,0.64867234,uid://A001/X5a4/X173,uid://A001/X5a4/X174,uid://A002/Xb7d0ee/X516a,The Almost Forgotten Local Ultra Luminous Infrared Galaxy: IRAS 13120-5453,S,TARGET,61.88235961140821,3.097545393266775,Y,0,"Merging and interacting galaxies, Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)",Galaxy evolution,2015.1.00287.S,94.08 .. 109.53,IRAS13120-5453,WKK 2031,198.77649,-55.15645,0.030761,True,True


Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,"Frequency ranges [4,2]",ALMA sanitized source name,NED source name,NED RA,NED Dec,Redshift,Line0 observed,Line1 observed
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,float64,str6,bytes30,float64,float64,float64,bool,bool
2015.1.00113.S,ARP220,233.7384041666667,23.50317777777778,36.627305952700226,53.02850549164822,[3],0.053279525709232764,3906.0797615918846,12m,,1215.648,2016-11-27,"[94.06..95.94GHz,3906.08kHz,921.4uJy/beam@10km/s,37.9uJy/beam@native, XX YY] U [95.94..97.81GHz,3906.08kHz,913.9uJy/beam@10km/s,37.9uJy/beam@native, XX YY] U [106.56..108.44GHz,3906.08kHz,930.9uJy/beam@10km/s,40.7uJy/beam@native, XX YY] U [107.94..109.81GHz,3906.08kHz,953.9uJy/beam@10km/s,42uJy/beam@native, XX YY]",10663.470009062945,XX YY,2015-10-18T17:29:47,"Scoville, Nick",ARP220_b_03_TE,"Torrey, Paul; Koda, Jin; Walter, Fabian; Murchikova, Lena; Hernquist, Lars; Sanders, David; Hayward, Christopher; Cox, Pierre; Barnes, Joshua; Thompson, Todd; Robertson, Brant; Vanden Bout, Paul; van der Werf, Paul; Decarli, Roberto; Genzel, Reinhard; Armus, Lee; Tacconi, Linda; Vlahakis, Catherine; Sheth, Kartik;",0.9138933291732583,0.019754150475718615,1.0218359,uid://A001/X2d6/X32,uid://A001/X2d6/X33,uid://A002/Xac2df7/X1b18,Arp 220 Nuclear Disks at 50 mas Resolution,S,TARGET,61.80306659404423,0.5478316056798604,Y,3,Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG),Galaxy evolution,2015.1.00113.S,94.06 .. 109.81,ARP220,ARP 220,233.73856,23.50314,0.018126,True,True
2015.1.00113.S,ARP220,233.73840416671834,23.50317777778901,36.627305952737856,53.028505491605166,[3],0.4838313038510857,3906.2872119731464,12m,,483.84,2017-11-19,"[94.06..95.94GHz,3906.29kHz,1.5mJy/beam@10km/s,60.9uJy/beam@native, XX YY] U [95.94..97.81GHz,3906.29kHz,1.5mJy/beam@10km/s,61.1uJy/beam@native, XX YY] U [106.56..108.44GHz,3906.29kHz,1.5mJy/beam@10km/s,66uJy/beam@native, XX YY] U [107.94..109.81GHz,3906.29kHz,1.5mJy/beam@10km/s,68.1uJy/beam@native, XX YY]",10664.597821039306,XX YY,2016-09-04T23:05:11,"Scoville, Nick",ARP220_d_03_TE,"Torrey, Paul; Koda, Jin; Walter, Fabian; Murchikova, Lena; Hernquist, Lars; Sanders, David; Hayward, Christopher; Cox, Pierre; Barnes, Joshua; Thompson, Todd; Robertson, Brant; Vanden Bout, Paul; van der Werf, Paul; Decarli, Roberto; Genzel, Reinhard; Armus, Lee; Tacconi, Linda; Vlahakis, Catherine; Sheth, Kartik;",1.4719337778903994,0.031906308730580625,1.9294773,uid://A001/X2d6/X48,uid://A001/X2d6/X49,uid://A002/Xb7a3f8/X9513,Arp 220 Nuclear Disks at 50 mas Resolution,S,TARGET,61.80304306329056,2.425079524681709,Y,3,Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG),Galaxy evolution,2015.1.00113.S,94.06 .. 109.81,ARP220,ARP 220,233.73856,23.50314,0.018126,True,True
2015.1.00167.S,Arp220,233.73862916664658,23.502911111112418,36.62696198379513,53.028239240472494,[3],0.5070505257793383,7812.613604801035,12m,,967.68,2017-12-15,"[94.81..96.68GHz,7812.61kHz,1.2mJy/beam@10km/s,50.8uJy/beam@native, XX YY] U [96.58..98.45GHz,7812.61kHz,1.2mJy/beam@10km/s,50.7uJy/beam@native, XX YY] U [106.81..108.68GHz,7812.61kHz,1.3mJy/beam@10km/s,55.1uJy/beam@native, XX YY] U [108.58..110.45GHz,7812.61kHz,1.3mJy/beam@10km/s

Project code,ALMA source name,ALMA RA,ALMA Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,Frequency ranges,ALMA sanitized source name,NED source name,NED RA,NED Dec,Redshift,Line0 observed,Line1 observed
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,GHz,,,degrees,degrees,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,object,str7,bytes30,float64,float64,float64,bool,bool
2013.1.00049.S,NGC_6240,253.24525416666663,2.400986,20.729013636056607,27.29104750337486,[3],1.7466723039466299,1952.8728015528832,12m,,2842.56,2016-08-07,"[94.71..96.59GHz,1952.88kHz,697.6uJy/beam@10km/s,28.8uJy/beam@native, XX YY] U [106.64..108.52GHz,1952.88kHz,719.2uJy/beam@10km/s,31.5uJy/beam@native, XX YY]",5394.400598303093,XX YY,2015-04-11T06:03:58,"Cicone, Claudia",NGC_6240_a_03_TE,"Papadopoulos, Padelis; Thi, Wing-Fai; Wagg, Jeff; Maiolino, Roberto; Zhang, Zhiyu; Feruglio, Chiara;",0.6976460758163437,0.021235647531750175,7.0027866,uid://A001/X197/X1d,uid://A001/X197/X1e,uid://A002/X9de499/X28d7,An Unprecedented ALMA View of the Massive Molecular Outflow in NGC 6240,S,TARGET,61.99891585886438,11.355323532320442,Y,0,"Outflows, jets, feedback",Active galaxies,2013.1.00049.S,"[array([94.71, 96.59]), array([106.64, 108.52])]",NGC6240,NGC 6240,253.24529,2.40093,0.02448,True,True
2015.1.01439.S,ngc6240,253.24525420002365,2.40098610000017,20.72901375247403,27.291047523549974,[3],0.49127181178248386,15625.497362928949,12m,,937.44,2018-04-21,"[93.27..95.14GHz,15625.50kHz,1.2mJy/beam@10km/s,49.4uJy/beam@native, XX YY] U [95.08..96.95GHz,15625.50kHz,1.2mJy/beam@10km/s,49.6uJy/beam@native, XX YY] U [105.07..107.05GHz,31250.99kHz,1.2mJy/beam@10km/s,50uJy/beam@native, XX YY] U [106.94..108.80GHz,15625.50kHz,1.2mJy/beam@10km/s,52.9uJy/beam@native, XX YY]",43054.99410637717,XX YY,2016-08-23T23:54:33,"Saito, Toshiki",ngc6240_b_03_TE,"Michiyama, Tomonari; Ando, Misaki; Kawabe, Ryohei; Iono, Daisuke; Ueda, Junko; Yamashita, Takuji;",1.1885420402544744,0.02519270787340361,2.0640316,uid://A001/X2fe/X727,uid://A001/X2fe/X728,uid://A002/Xb7189f/X84e2,UNVEILING MERGER-INDUCED SHOCKS IN MOLECULAR HYDROGEN EMITTING GALAXIES (MOHEGs),S,TARGET,62.3536086848821,2.435858225616976,Y,0,"Merging and interacting galaxies, Galaxy chemistry",Galaxy evolution,2015.1.01439.S,"[array([93.27, 95.14]), array([95.08, 96.95]), array([105.07, 107.05]), array([106.94, 108.8 ])]",NGC6240,NGC 6240,253.24529,2.40093,0.02448,True,True


<span style="color:red"> - The `runQueryWithLines` class method is missing galaxies IRAS 17208-004</span>

<span style="color:red"> - It's also a tad slower, probably due to looping over lines then spectral windown rather than spectral windows then lines. I'm not sure exactly why this is </span>